# Processaimento de imagem - SCC5830

## Aluno:
 - Victor Akihito Kamada Tomita | 10692082 | akihito012@usp.br | CCMC

obs: este notebook foi desenvolvido no google colab, por favor utilize o ambiente do colab para testar este notebook

[link da apresentação](https://www.canva.com/design/DAE-o99pj7Q/xQMNOmEmnkHKLrAeJkeOKg/view?utm_content=DAE-o99pj7Q&utm_campaign=designshare&utm_medium=link&utm_source=homepage_design_menu)

---
# Análise de sentimentos multimodal

---

Neste notebook serão utilizados os modelos unimadais de análise de sentimentos de texto e imagem, que foram treinados pelos notebooks <code>Análise de sentimentos em texto</code> e <code>Análise de sentimentos em faces</code>, para gerar embeddings das faces e dos textos transcritos.

A partir disso, faremos co-traning do modelo multimodal de fusão híbrida por meio do treinamento de concordância dos modelos unimodais. 

Por fim, para a classificação final aplicaremos o modelo multimodal para casos de discordância entre modelos unimodais, e para o resto dos casos utilizaremos a condordância dos modelos unimodais

---


In [ ]:
%%capture
!pip install transformers
!pip install ktrain
!pip install gdown
!pip install --upgrade gdown

In [ ]:
%%capture
!pip install -U sentence-transformers

In [ ]:
%%capture
!gdown 1JHLBzX5tbrjetisIIRUcSpYCj5Kb8etS
!gdown 1wJlyjjUniZ8lmA6QRoMXSoe6Oy5MLIRE

In [ ]:
%%capture
!gdown 1nomOVu9guM1nWji_Ngt5ot0Twy0kaYID

In [ ]:
%%capture
!mkdir models
!unzip text_model.zip -d ./models/
!unzip sentiment_model_1.zip -d ./models/
!unzip sentiment_model_2_ktrain_EXP4.zip -d ./models/

In [ ]:
%%capture
!pip install chitra
!pip install -U opencv-python

In [ ]:
%%capture
!gdown 1IVDppeuLIRESFVjELv6cGO6ExuCUFxwr
!gdown 1TovneWlWJsjTr95m7C0284bcephlixj2
!unzip models_for_face_extraction_celeb.zip -d models/face_ext

In [ ]:
%%capture
!gdown 1N7752EdJQk_UbCxvwyVbnFHE5jHuDmE5
!unzip models_NN_for_face_extraction_celab.zip 

In [ ]:
!gdown 1cAZh3-CwidLM3wccFZOwC5IktTdqoTbu

Downloading...
From: https://drive.google.com/uc?id=1cAZh3-CwidLM3wccFZOwC5IktTdqoTbu
To: /content/knnRegression_5.sav
100% 57.8M/57.8M [00:00<00:00, 114MB/s]


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.layers import Input, Dense, Add
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from chitra.image import Chitra
import tensorflow as tf
import numpy as np
import ktrain
import keras
import pickle
import cv2


In [ ]:
%%capture
bert_model = SentenceTransformer('distiluse-base-multilingual-cased')

In [ ]:
## sentiment model ##
image_model_predictor = ktrain.load_predictor('/content/models/sentiment_model_2_ktrain_EXP4')
image_model = image_model_predictor.model

In [ ]:
### for feature extraction (BOWV) ###
feature_extractor = cv2.SIFT_create()
kmeans = pickle.load(open('/content/kmeans_model_for_BOVW_clustering_celeb.sav', 'rb'))
num_visual_words = 512

### for bounding box generation ###
grad_boost = pickle.load(open('./models/face_ext/models/gradient_model.sav', 'rb'))
knnRegressor = pickle.load(open('./models/face_ext/models/knnRegression.sav', 'rb'))
knn_temp = pickle.load(open('knnRegression_5.sav', 'rb'))
simpleNN = pickle.load(open('./models/face_ext/models/simpleNN.sav', 'rb'))

### model for face extraction using neural nets ###
simple_nn = keras.models.load_model('/content/models/simple_dense')
simple_nn_skip = keras.models.load_model('/content/models/dense_with_skip')

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator MiniBatchKMeans from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator GradientBoostingRegressor from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator DummyRegressor from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at

In [ ]:
## text model ##
text_model = keras.models.load_model('/content/models/text_model')

In [ ]:
## definição de possíveis variaveis em uso ##


## t_hold_face - lista para filtragem de sentimento em face
## primeiro valor é o threshold para definição de negativo
## segundo valor é o threshold para definição de positivo
### obs - para o modelo classificar ele só precisa de uma precisão de 0.3
t_hold_face = [0.34,0.34]

## t_hold_text - lista para filtragem de sentimento em face
## primeiro valor é o threshold para definição de negativo
## segundo valor é o threshold para definição de positivo
### obs - para o modelo classificar ele só precisa de uma precisão de 0.3
t_hold_text = [0.2, 0.65]

Definindo algumas funções para extração de embeddings/classificação do modelo de texto

In [ ]:
from keras import backend as K
import cv2

In [ ]:
def get_embedding_text(text, return_sentiment=False):
  global text_model, bert_model

  inp = text_model.input
  encoded_text = bert_model.encode([text])
  functor1 = K.function([inp], text_model.layers[-2].output)

  if return_sentiment:
    return functor1([encoded_text])[0], text_model.predict([encoded_text])[0][1]
  return functor1([encoded_text])[0]

def get_sentiment_text(text):
  global text_model, bert_model
  encoded_text = bert_model.encode([text])
  return text_model.predict([encoded_text])[0][1]


definindo algumas funções para extração de embeddings/classificação do modelo de face

In [ ]:
## t_hold - lista para filtragem de sentimento
## primeiro valor é o threshold para definição de negativo
## segundo valor é o threshold para definição de positivo

def get_sentiment_val(sent_dist):
  global t_hold_face
  cur_sentiment =  np.argmax(sent_dist)
  if cur_sentiment == 0 and t_hold_face[0] <= sent_dist[0]: ## threshold de sentimento negativo
    return 0
  if cur_sentiment == 2 and t_hold_face[1] <= sent_dist[2]: ## threshold de sentimento positivo
    return 2
  return 1



def get_embedding_image(gray_face, return_sentiment=False):
  global image_model
  inp = image_model.input
  functor1 = K.function([inp], image_model.layers[-2].output)
  if return_sentiment:
    return functor1([y])[0], image_model.predict([y])[0]

  return functor1([y])[0]

def get_sentiment_image(imgPath):
  global image_model

  img = cv2.imread(imgPath)
  res = cv2.resize(img, dsize=(244, 244), interpolation=cv2.INTER_CUBIC)
  y = res[None, :]
  print( y.shape)

  return image_model.predict([y])[0]

In [ ]:
img_feat_size = 64
text_feat_size = 64

In [ ]:
emotion_labels = {
    0:'negative',
    1:'neutral',
    2:'positive'
}

# Definição de pipeline para extração de vídeo e extração de características
---
Para que possamos extrair o sentimento de um vídeo, precisamos inicialmente do vídeo. Em função disso, utilizo o *Pytube* para extrair os vídeos do *Youtube*. O problema disso é que por vezes o *Youtube* troca a forma de reponder às requisições e o *Pytube* é uma biblioteca que demora para ser atualizada. Em função disso, é comum ocorrer erros nesta parte de <code>Download do vídeo</code>, uma vez que não depende de mim, mas sim do *Pytube*. Qualquer problema envie um email para mim

In [ ]:
# gets the string time as HH:MM:SS and return in seconds
def string_time_int(str_time):
  segundos = int(str_time[-2:])
  segundos += 60 * int(str_time[-5:-3])
  segundos += 3600 * int(str_time[:-6])
  return segundos
def process_caption(video, lang='a.en'):
  '''
    extract the information from the caption and saves in a caption struct 
  '''
  has_caption = False
  print(video.captions)
  for cap in video.captions:
    if cap.code == lang:
      has_caption = True

  if not has_caption:
    print("caption of lang : " + lang + " not found")

  video_len = video.length

  captions  = video.captions[lang]
  captions_str = str(captions.generate_srt_captions())
  list_captions = []
  line_counter = 1
  for line in captions_str.split('\n'):
      # time type
      if (line_counter + 2) % 4  == 0:
          line_aux = line.split(" --> ")
          time_s = string_time_int(line_aux[0][:-4])
          time_e = string_time_int(line_aux[1][:-4])
      # comment type
      if (line_counter + 1) % 4  == 0 and video_len >= time_e:  
          list_captions.append((line, time_s, time_e))
      line_counter += 1

  return list_captions

In [ ]:
%%capture
!python -m pip install git+https://github.com/Vakihito/pytube

In [ ]:
from google.colab.patches import cv2_imshow
from pytube.helpers import safe_filename
from keras.models import load_model
from keras import backend as K
from statistics import mode
from pytube import YouTube
import cv2
import os

In [ ]:
def my_preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

def predict_text(text):
    global predictor_bert_lex, t_hold_text
    polarity_text = get_sentiment_text(text)

    color = np.asarray((255., 255., 255.))

    val_mult = 1
    if polarity_text < t_hold_text[0]:
        polarity_text = 1 - abs(polarity_text)
        color = (polarity_text) * np.asarray((255., 0., 0.))
        val_mult = -1
    elif polarity_text > t_hold_text[1]:
        color = abs(polarity_text) * np.asarray((0., 255., 0.))
    else : 
        polarity_text = 0.0

    return color, polarity_text * val_mult

In [ ]:
# hyper-parameters for bounding boxes shape
frame_window = 10
emotion_offsets = (20, 40)

## Download do vídeo
---
Nesta seção fazemos o download do vídeo para que possamos fazer a análise dele, nesta parte podem ocorrer erros por causa da forma que o Youtube responde à requisições ( dependo do Pytube para lidar com as requisições e como supracitado o Pytube não é uma biblioteca muito boa, por isso tive que fazer um fork da blibioteca para fazer bug fix dela 🤕 ) 

---

In [ ]:
from pytube import YouTube

In [ ]:
vid_id = "zrdNHv4hs4E"
selected_lang = 'en'
res = 720
fps_my = 30

In [ ]:
emotion_target_size = (244, 244)
emotion_window = []
# starting video streaming




url = "https://www.youtube.com/watch?v=" + vid_id
print("downloading the video from the ulr : ", url)
video = YouTube(url)

min_res = res
itag_max = -1

for stream in video.streams:
    if stream.resolution and stream.mime_type == "video/mp4":
      current_res = int(stream.resolution[:-1])
      current_fps = int(stream.fps)
      if res == current_res and current_fps==fps_my:
        itag_max = stream.itag
        fps_my =  float(current_fps) 

print(str(itag_max))
video.streams.get_by_itag(str(itag_max)).download()

dirname = './'
video_path = os.path.join(dirname, safe_filename(video.title) + '.mp4')


#documentation: https://pypi.org/project/pafy/
video_capture = cv2.VideoCapture(video_path)


captions_list = process_caption(video, selected_lang)
captions_list.sort(key=lambda x: x[1])


time = 0
frame_counter = 0
caption_counter = 0

downloading the video from the ulr :  https://www.youtube.com/watch?v=zrdNHv4hs4E
136
{'en': <Caption lang="English" code="en">, 'a.en': <Caption lang="English (auto-generated)" code="a.en">}


In [ ]:
video_dict_all = {
  'frame' : [],
  'time' : [],
  'caption' : [],
  'caption_polarity' : [],
  'caption_emb' : [],
  'face' : [],
  'face_polarity' : [],
  'face_emb' : []
}

In [ ]:
def fill_dict(frame,time,
              caption, caption_polarity, caption_emb,
              face, face_polarity, face_emb):
  video_df_all = {
    'frame' : [frame],
    'time' : [time],
    'caption' : caption,
    'caption_polarity' : caption_polarity,
    'caption_emb' : caption_emb,
    'face' : face,
    'face_polarity' : face_polarity,
    'face_emb' : face_emb
  }

  if len(caption) == 0:
    video_df_all['caption'] = ['-1']
    video_df_all['caption_polarity'] = [0]
    video_df_all['caption_emb'] = [np.zeros(text_feat_size)]

  if len(face) == 0:
    video_df_all['face'] = ['-1']
    video_df_all['face_polarity'] = [0]
    video_df_all['face_emb'] = [np.zeros(img_feat_size)]
  
  if len(face) >= 2:
    for i in range(len(face) - 1):
      video_df_all['frame'].append(video_df_all['frame'][-1])
      video_df_all['time'].append(video_df_all['time'][-1])
      video_df_all['caption'].append(video_df_all['caption'][-1])
      video_df_all['caption_polarity'].append(video_df_all['caption_polarity'][-1])
      video_df_all['caption_emb'].append(video_df_all['caption_emb'][-1])

  return video_df_all

In [ ]:
captions_list_size = len(captions_list)

def return_text(time, counter):
  temp_text = ""
  for i in range(counter, captions_list_size):
    if time >= captions_list[i][1] and time <= captions_list[caption_counter][2]:
      temp_text =  captions_list[i][0] + " "
    elif captions_list[i][1] > time:
      return  temp_text
  return temp_text

## Extração de características e de classificação
--- 

In [ ]:
path_images_unimodel = './images_unimodel/'
path_images_multimodel = './images_multimodel/'

os.mkdir(path_images_unimodel)
os.mkdir(path_images_multimodel)

display_image = False
save_image = True

In [ ]:
font_scale_cur = 1.5

In [ ]:
def get_bowv_from_img(rgb_img):
  inicial_img_shape = rgb_img.shape
  resize_img = cv2.resize(rgb_img, (480,480))
  _, sift_vectors = feature_extractor.detectAndCompute(resize_img, None)

  hist = kmeans.predict(sift_vectors)
  hist = np.histogram(hist, 512)[0]

  return hist, resize_img, inicial_img_shape

def increase_by(val, to_inc):
  val += val * to_inc
  return min(val, 480)  

def decrese_by(val, to_inc):
  val -= val * to_inc
  return max(val, 0)

def expand_box(box, by_p=0.2):
  box[0] = decrese_by(box[0], by_p)
  box[1] = decrese_by(box[1], by_p)

  box[2] = increase_by(box[2], by_p)
  box[3] = increase_by(box[3], by_p)

def get_prediction_for_image(rgb_img, cur_classier, exp_by=0.0):

  bowv, resized_img, inicial_img_shape = get_bowv_from_img(rgb_img)
  bowv = bowv/np.sum(bowv)
  position_pred = cur_classier.predict(np.array([bowv]))[0] * 480
  
  expand_box(position_pred, exp_by)
  
  image_pred = Chitra(np.array(resized_img), bboxes=position_pred, labels='face')
  image_pred.resize_image_with_bbox((inicial_img_shape[1], inicial_img_shape[0]))

  coords_boxes = [image_pred.bboxes[0].x1, image_pred.bboxes[0].y1, image_pred.bboxes[0].x2, image_pred.bboxes[0].y2]
  coor_x1x2y1y2 = [image_pred.bboxes[0].x1, image_pred.bboxes[0].x2, image_pred.bboxes[0].y1, image_pred.bboxes[0].y2]
  
  coords_boxes[0] = coords_boxes[0]
  coords_boxes[1] = coords_boxes[1]
  coords_boxes[2] = coords_boxes[2] - coords_boxes[0]
  coords_boxes[3] = coords_boxes[3] - coords_boxes[1]

  coords_boxes = [int(cur_coord) for cur_coord in coords_boxes]
  coor_x1x2y1y2 = [int(cur_coord) for cur_coord in coor_x1x2y1y2]

  return coords_boxes, coor_x1x2y1y2, image_pred


def get_prediction_for_coord_skip(rgb_img, exp_by=0.0):

  bowv, resized_img, inicial_img_shape = get_bowv_from_img(rgb_img)
  bowv = bowv/np.sum(bowv)
  simple_nn_skip_pred = simple_nn_skip.predict(np.array([bowv]))[0] * 480


  simple_nn_skip_pred = np.array(simple_nn_skip_pred)
  expand_box(simple_nn_skip_pred, exp_by)

  skip_nn_img = Chitra(np.array(resized_img), bboxes=simple_nn_skip_pred, labels='face')
  skip_nn_img.resize_image_with_bbox((inicial_img_shape[1], inicial_img_shape[0]))

  coords_boxes = [skip_nn_img.bboxes[0].x1, skip_nn_img.bboxes[0].y1, skip_nn_img.bboxes[0].x2, skip_nn_img.bboxes[0].y2]
  coor_x1x2y1y2 = [skip_nn_img.bboxes[0].x1, skip_nn_img.bboxes[0].x2, skip_nn_img.bboxes[0].y1, skip_nn_img.bboxes[0].y2]


  coords_boxes[0] = coords_boxes[0]
  coords_boxes[1] = coords_boxes[1]
  coords_boxes[2] = coords_boxes[2] - coords_boxes[0]
  coords_boxes[3] = coords_boxes[3] - coords_boxes[1]

  coords_boxes = [int(cur_coord) for cur_coord in coords_boxes]
  coor_x1x2y1y2 = [int(cur_coord) for cur_coord in coor_x1x2y1y2]

  return coords_boxes, coor_x1x2y1y2, skip_nn_img


def draw_bounding_box(face_coordinates, image_array, color):
    x, y, w, h = face_coordinates
    cv2.rectangle(image_array, (x, y), (x + w, y + h), color, 2)
  

def draw_text(coordinates, image_array, text, color, x_offset=0, y_offset=0,
                                                font_scale=2, thickness=2):
    x, y = coordinates[:2]
    cv2.putText(image_array, text, (x + x_offset, y + y_offset),
                cv2.FONT_HERSHEY_SIMPLEX,
                font_scale, color, thickness, cv2.LINE_AA)


In [ ]:
!rm -r /content/images_unimodel
!mkdir images_unimodel

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
img_counter = 0
last_text = "**!!"
# Coordenadas 
# 1 - distancia da borda lateral esquerda 
# 2 - distancia da borda superior 
#

while True:
    success , bgr_image = video_capture.read()
    
    if not success:
        break
    gray_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)

    # faces_bb, coor_x1x2y1y2, image_pred  = get_prediction_for_image(rgb_image, knn_temp, exp_by=0.2)
    faces_bb, coor_x1x2y1y2, image_pred  = get_prediction_for_coord_skip(rgb_image, exp_by=0.0)
    time = frame_counter/fps_my 

    caption = []
    caption_polarity = []
    caption_emb = []

    cur_text = return_text(time, caption_counter)
    if len(cur_text) > 0:
        # if last_text != cur_text and last_text == cur_text[:len(last_text)]:
        #   cur_text = cur_text[len(last_text):]
        #   last_text = cur_text

        last_text = cur_text 
        color_caption, caption_polarity = predict_text(cur_text)
        caption = [cur_text]
        caption_polarity = [caption_polarity]
        caption_emb = [get_embedding_text(cur_text)]
        
        print("caption polarity :: ", caption_polarity)
        print("caption sentence :: ", cur_text)
        print("caption color :: ", color_caption)


        draw_text(np.array([10,min_res - 30,50,50]), rgb_image, cur_text,color_caption,font_scale=0.3 * font_scale_cur, thickness=1)
        while caption_counter < len(captions_list) - 1 and time >= captions_list[caption_counter][2]:  
            caption_counter += 1

    face = []
    face_polarity = []
    face_emb = []
    
    

    x1, x2, y1, y2 = coor_x1x2y1y2
    gray_face = gray_image[y1:y2, x1:x2]
    try:
        gray_face = cv2.resize(gray_face, dsize=(244, 244), interpolation=cv2.INTER_CUBIC)
    except cv2.error:
        print('error : on resize')
        continue

    gray_face = cv2.cvtColor(gray_face, cv2.COLOR_GRAY2BGR)
    y = gray_face[None, :]
    emotion_prediction = image_model.predict([y])[0]

    emotion_label_arg = np.argmax(emotion_prediction)
    emotion_probability = emotion_prediction[emotion_label_arg]
    print("face_coods :: ", coor_x1x2y1y2)
    print("face_model_prediction :: ", emotion_prediction)
    print("emotion_label_arg :: ", emotion_label_arg)
    print("emotion_probability :: ", emotion_probability)
    emotion_text = emotion_labels[emotion_label_arg]
    emotion_window.append(emotion_text)

    if len(emotion_window) > frame_window:
        emotion_window.pop(0)
    try:
        emotion_mode = mode(emotion_window)
    except:
        continue

    ### threshold wall
    # ysnp = you shall not pass
    ysnp = True
    sentiment = 'neutral'

    if ysnp:
        senti_multi = 0
        if emotion_text == 'negative':
            senti_multi = -1
            color = emotion_probability * np.asarray((255, 0, 0))
        elif emotion_text == 'positive':
            senti_multi = 1
            color = emotion_probability * np.asarray((0, 255, 0))
        else:
            senti_multi = 0
            color = emotion_probability * np.asarray((0, 0, 255))

        color = color.astype(int)
        color = color.tolist()

        draw_bounding_box(faces_bb, rgb_image, color)
        draw_text(faces_bb, rgb_image, emotion_text,color, 0, -45, 1, 1)
        
        print('face polarity: ', emotion_probability * senti_multi,emotion_text)

        face.append('img' + str(img_counter))
        face_polarity.append(emotion_probability * senti_multi)
        face_emb.append(get_embedding_image(gray_face) )

        img_counter += 1
   



    dict_temp= fill_dict(frame_counter, time,
                        caption, caption_polarity, caption_emb,
                        face, face_polarity, face_emb)       
    for key in dict_temp.keys():
      video_dict_all[key] += dict_temp[key]
    
    print(" frame_counter :: ", frame_counter)
    frame_counter += 1

    bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
    if display_image:
      cv2_imshow( bgr_image)
    if save_image:
      cv2.imwrite(path_images_unimodel + 'uni_' + str(frame_counter) + '.jpg', bgr_image)
    print("_" * 50)
video_capture.release()
cv2.destroyAllWindows()


Computing label assignment and total inertia
caption polarity ::  [0.980513870716095]
caption sentence ::  So today I'm really glad 
caption color ::  [  0.         250.03103703   0.        ]
face_coods ::  [66, 339, 80, 627]
face_model_prediction ::  [0.06318469 0.36777797 0.5690374 ]
emotion_label_arg ::  2
emotion_probability ::  0.5690374
face polarity:  0.5690373778343201 positive
 frame_counter ::  0
__________________________________________________
Computing label assignment and total inertia
caption polarity ::  [0.980513870716095]
caption sentence ::  So today I'm really glad 
caption color ::  [  0.         250.03103703   0.        ]
face_coods ::  [66, 339, 79, 628]
face_model_prediction ::  [0.06085492 0.38467836 0.5544667 ]
emotion_label_arg ::  2
emotion_probability ::  0.5544667
face polarity:  0.554466724395752 positive
 frame_counter ::  1
__________________________________________________
Computing label assignment and total inertia
caption polarity ::  [0.9805138707

## Desenvolvimento do modelo multimodal

---

In [ ]:
import pandas as pd

In [ ]:
video_df = pd.DataFrame(video_dict_all)
video_df.head()

,frame,time,caption,caption_polarity,caption_emb,face,face_polarity,face_emb
0,0,0.000000,So today I'm really glad,0.980514,"[0.0, 0.36272267, 0.0, 0.89719254, 0.80055064,...",img0,0.569037,"[0.0, 0.0, 0.0, 0.0, 0.0, 4.510489, 0.0, 0.0, ..."
1,1,0.033333,So today I'm really glad,0.980514,"[0.0, 0.36272267, 0.0, 0.89719254, 0.80055064,...",img1,0.554467,"[0.0, 0.0, 0.0, 0.0, 0.0, 4.7294016, 0.0, 0.0,..."
2,2,0.066667,So today I'm really glad,0.980514,"[0.0, 0.36272267, 0.0, 0.89719254, 0.80055064,...",img2,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 13.155323, 0.0, 0.0,..."
3,3,0.100000,So today I'm really glad,0.980514,"[0.0, 0.36272267, 0.0, 0.89719254, 0.80055064,...",img3,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 13.922622, 0.0, 0.0,..."
4,4,0.133333,So today I'm really glad,0.980514,"[0.0, 0.36272267, 0.0, 0.89719254, 0.80055064,...",img4,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 14.690789, 0.0, 0.0,..."


### Gerando DataFrames de concordância

In [ ]:
def concord_bool(a,b):

  multi = a * b
  if multi >= 0:
    return True
  return False

def concord_val(a,b):
  (pa , pb) = (a,b)
  if abs(a) < 0.3:
    pa = 0
  if abs(b) < 0.3:
    pb = 0
  multi = pa * pb
  if multi >= 0:
    if pa < 0 or pb < 0:
      return 'negative'
    if pa > 0 or pb > 0:
      return 'positive'
  return 'neutral'

def return_dict():
  dict_aux = {
    'frame' : [],
    'time' : [],
    'caption' : [],
    'caption_polarity' : [],
    'caption_emb' : [],
    'face' : [],
    'face_polarity' : [],
    'face_emb' : [],
    'intent' : []
  }
  return dict_aux

In [ ]:
def get_concord_embedding():
  global video_df
  couter = 0
  df_train = return_dict()
  df_test = return_dict()
  concord = []
  for i , row in video_df.iterrows():
    if concord_bool(row['face_polarity'],row['caption_polarity']):
      for key in row.keys():
        df_train[key].append(row[key])
      df_train['intent'].append(concord_val(row['face_polarity'],row['caption_polarity']))
      concord.append(1)
    else:
      for key in row.keys():
        df_test[key].append(row[key])
      df_test['intent'].append(concord_val(row['face_polarity'],row['caption_polarity']))
      concord.append(0)
  video_df['consent'] = concord
  return df_train, df_test

In [ ]:
(df_train, df_test) =  get_concord_embedding()
df_train = pd.DataFrame(df_train)
df_test = pd.DataFrame(df_test)

## Definindo o modelo multimodal
---
A partir deste ponto defino o modelo multimodal de fusão, em específico utilizo o modelo multimodal de fusão por atenção em relação às modalidades

In [ ]:
import tensorflow as tf
ACCURACY_THRESHOLD = 0.95
class acc_callback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy') > ACCURACY_THRESHOLD):   
            self.model.stop_training = True

In [ ]:
from tensorflow.keras.layers import Dot,Activation,Dense, Input, concatenate, multiply, average, subtract, add, Dropout, Lambda, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf


def multimodal_text_image(num_classes,size_1=img_feat_size,size_2=text_feat_size,operator='concatenate',verbose=0):

  # fusion_dim = X_1.shape[1]+X_2.shape[1]
  fusion_dim = size_1

  inp1 = Input(shape=size_1)
  inp2 = Input(shape=size_2)

  l1 = Dense(fusion_dim, activation='relu')(inp1)
  l2 = Dense(fusion_dim, activation='relu')(inp2)

  if(operator=='att'):
    visual_embd = Lambda(lambda x: tf.keras.backend.l2_normalize(x, axis=-1))(l1) # (bs, ndim)
    average_seq = Lambda(lambda x: tf.keras.backend.l2_normalize(x, axis=-1))(l2) # (bs, ndim)
    scalar_visual = Dense(1)(visual_embd) # (bs, 1)
    scalar_text = Dense(1)(average_seq) # (bs, 1)
    scalars = concatenate([scalar_visual, scalar_text], name='concat')  # (bs, 2)

    # # Step 2. Normalize weights - softmax
    alphas = Activation('softmax')(scalars) # (bs, 2)

    # Step 3. Weighted average
    visual_embd_2 = Lambda( lambda x: tf.keras.backend.expand_dims(x) ) (visual_embd) # (bs, ndim, 1)
    average_seq_2 = Lambda( lambda x: tf.keras.backend.expand_dims(x) )(average_seq) # (bs, ndim, 1)
    features = concatenate([visual_embd_2, average_seq_2], name='concat_feats') # (bs, ndim, 2)
    w = Dot(axes=[-1, -1])([alphas, features]) # (bs, ndim)

  w = Dropout(0.5)(w)
  # fusion_layer = Dense(fusion_dim, activation='relu')(w)
  fusion_layer = w

  if (operator == 'att_labels'): # nm: new
    output = Dense(1)(fusion_layer)  # (batch_size, nb_labels, 1)  
    output = Lambda(lambda x: tf.keras.backend.squeeze(x, axis=-1))(output)  # (batch_size, nb_labels)
    output = Activation('softmax')(output)  # (batch_size, nb_labels)    
  else:
    output = Dense(num_classes,activation='softmax')(fusion_layer)

  model = Model(inputs=[inp1, inp2], outputs=output)

  model.compile(optimizer='adam', loss='categorical_crossentropy', run_eagerly=True)
  model.summary()

  return model, fusion_layer



In [ ]:
import keras 
from tqdm.notebook import tqdm

target = 'intent'

merging_layers = ['autoencoder', 'att_labels','att','concatenate','add','subtract','average','multiply']

num_classes = len(df_train[target].unique())
operator = "att"
# Gerando os modelos
my_model, _ = multimodal_text_image(num_classes,operator=operator)




# merging_layers = ['att', 'att_labels'] # test
results = []



(df_data_train, df_data_test) = (df_train, df_test)

total_rows = len(df_data_train[target])
weights_proportion = {
    0: 1, 1: 1, 2: 1
}


# modalidade da imagem
X_1 = np.array(df_data_train['face_emb'].to_list())
# modalidade do texto
X_2 = np.array(df_data_train['caption_emb'].to_list())

Y = np.array(pd.get_dummies(df_data_train[target]))

num_classes = len(df_data_train[target].unique())

# modalidades imagem e texto do teste
X_1_test = np.array(df_data_test['face_emb'].to_list())
X_2_test = np.array(df_data_test['caption_emb'].to_list())

acc_call = acc_callback()


history = my_model.fit([X_1,X_2], Y,
                  epochs=50,
                  batch_size=16,
                  shuffle=True,verbose=True)


Model: "model_825"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 64)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 64)           4160        ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 64)           4160        ['input_2[0][0]']                
                                                                                          

In [ ]:
dict_y_pred = {
    0 : 'negative', 
    1 : 'neutral',
    2 : 'positive'
}

if len(X_1_test) != 0:
  probs = my_model.predict([X_1_test,X_2_test])
  y_pred = np.argmax(probs,axis=1)
  y_pred_value = [dict_y_pred[pred] for pred in y_pred]
else:
  y_pred_value = []

In [ ]:
index = video_df[video_df['consent'] == 0].index

In [ ]:
y_pred_value_size = len(y_pred_value)

tuple_idx_val = []

for i in range(y_pred_value_size):
  tuple_idx_val.append((index[i], y_pred_value[i]))

In [ ]:
video_consent_df = video_df[video_df['consent'] == 1]

for idx, row in video_consent_df.iterrows():
  tuple_idx_val.append((idx, concord_val(row['caption_polarity'],row['face_polarity']) ))

In [ ]:
tuple_idx_val.sort(key=lambda x:x[0])
final_pred = [i[1] for i in tuple_idx_val]


video_df['final_pred'] = final_pred

In [ ]:
video_df

,frame,time,caption,caption_polarity,caption_emb,face,face_polarity,face_emb,consent,final_pred
0,0,0.000000,So today I'm really glad,0.980514,"[0.0, 0.36272267, 0.0, 0.89719254, 0.80055064,...",img0,0.569037,"[0.0, 0.0, 0.0, 0.0, 0.0, 4.510489, 0.0, 0.0, ...",1,positive
1,1,0.033333,So today I'm really glad,0.980514,"[0.0, 0.36272267, 0.0, 0.89719254, 0.80055064,...",img1,0.554467,"[0.0, 0.0, 0.0, 0.0, 0.0, 4.7294016, 0.0, 0.0,...",1,positive
2,2,0.066667,So today I'm really glad,0.980514,"[0.0, 0.36272267, 0.0, 0.89719254, 0.80055064,...",img2,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 13.155323, 0.0, 0.0,...",1,positive
3,3,0.100000,So today I'm really glad,0.980514,"[0.0, 0.36272267, 0.0, 0.89719254, 0.80055064,...",img3,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 13.922622, 0.0, 0.0,...",1,positive
4,4,0.133333,So today I'm really glad,0.980514,"[0.0, 0.36272267, 0.0, 0.89719254, 0.80055064,...",img4,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 14.690789, 0.0, 0.0,...",1,positive
...,...,...,...,...,...,...,...,...,...,...
399,399,13.300000,-1,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",img399,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 12.632889, 0.0, 0.0,...",1,neutral
400,400,13.333333,-1,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",img400,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 11.023514, 0.0, 0.0,...",1,neutral
401,401,13.366667,-1,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",img401,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 4.341114, 0.0, 0.0, ...",1,neutral
402,402,13.400000,-1,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",img402,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 5.1336637, 0.0, 0.0,...",1,neutral


In [ ]:
def draw_plus_sign(img, center=(10,10) ):
  start_point = (center[0] - 5, center[1] - 2)
  end_point = (center[0] + 5, center[1] + 2)
  img_r = cv2.rectangle(img, start_point, end_point, (0,255,0), -1)

  start_point = (center[0] - 2, center[1] - 5)
  end_point = (center[0] + 2, center[1] + 5)
  img_r = cv2.rectangle(img_r, start_point, end_point, (0,255,0), -1)

  return img_r

def draw_minus_sign(img, center=(10,10)):
  start_point = (center[0] - 5, center[1] - 2)
  end_point = (center[0] + 5, center[1] + 2)
  img_r = cv2.rectangle(img, start_point, end_point, (255,0,0), -1)

  return img_r

In [ ]:
time_arr =video_df['time'].to_list()
final_pred_arr = video_df['final_pred'].to_list()
size_total = len(time_arr)

In [ ]:
tuple_time_pred = [(time_arr[i], final_pred_arr[i]) for i in range(size_total)]


In [ ]:
### re inicializando as variáveis.

video_capture = cv2.VideoCapture(video_path)
time = 0
frame_counter = 0
caption_counter = 0
counter_tuple_total = 0 

In [ ]:
save_image = True
display_image = False

### loop final de classificação
---
Neste loop são criadas as imagens finais do pipeline, aqui são criadas as fotos classificadas com o modelo multimodal de fusão e concordância entre as modalidades unimodais.

In [ ]:
while True:
    success , bgr_image = video_capture.read()
    
    if not success:
        break
    gray_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)

    time = frame_counter/fps_my 



    if time >= captions_list[caption_counter][1] and time <= captions_list[caption_counter][2]:
        color_caption = (255,255,255)
        draw_text(np.array([10,min_res - 30,50,50]), rgb_image, captions_list[caption_counter][0],
                  color_caption,font_scale=0.3 * font_scale_cur, thickness=1)
        if caption_counter < len(captions_list) - 1 and time >= captions_list[caption_counter + 1][1]:  
            caption_counter += 1

    if time >= tuple_time_pred[counter_tuple_total][0]:
        if (tuple_time_pred[counter_tuple_total][1] == 'positive') :
          rgb_image = draw_plus_sign(rgb_image)
        elif (tuple_time_pred[counter_tuple_total][1] == 'negative'):
          rgb_image = draw_minus_sign(rgb_image)

        if counter_tuple_total < len(tuple_time_pred) - 1 and time >= tuple_time_pred[counter_tuple_total + 1][0]:  
            counter_tuple_total += 1
    
    
    frame_counter += 1

    bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)

    if display_image:
      cv2_imshow( bgr_image)
    if save_image:
      cv2.imwrite(path_images_multimodel + 'uni_' + str(frame_counter) + '.jpg', bgr_image)


video_capture.release()
cv2.destroyAllWindows()

In [ ]:
%%capture
!zip -r images_multimodel.zip /content/images_multimodel
!zip -r images_unimodel.zip /content/images_unimodel


In [ ]:
from google.colab import files
files.download('images_multimodel.zip') 
files.download('images_unimodel.zip') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>